## 5.7 誤差逆伝播法の実装

### 5.7.1 ニューラルネットワークの学習の全体像

ニューラルネットワークの学習とは、重みとバイアスを訓練データに適応することであり、次の4つのステップで行う。

#### ステップ1（ミニバッチ）
　訓練データの中からランダムに一部のデータを選び出す。
 
#### ステップ2（勾配の算出）
　各重みパラメータに関する損失関数の勾配を求める。

#### ステップ3（パラメータの更新）
　重みパラメータを勾配方向に微笑量だけ更新する。

#### ステップ4（繰り返す=ミニバッチサイズ分まで）
　ステップ1、ステップ2、ステップ3を繰り返す。
 
#### ステップ5（繰り返す=ステップ1~4=epoh）
　訓練データの中からランダムに一部のデータを選び出す。
 
前節までは、ステップ2の勾配を求めるために、数値微分を用いていたが、計算時間がかかる。
<br>誤差逆伝播を用いれば、高速に効率良く勾配を求めることができる。



### 5.7.2 誤差逆伝播に対応したニューラルネットワークの実装
ここでは、2層のニューラルネットを実装する。また、前章からの主な変更点は、<b>レイヤ</b>を使用していることである。レイヤを使用することで、認識結果を得る処理や勾配を求める処理がレイヤの伝播だけで達成することができるようになる。
<br>（この本では重みを持つ層を1層、2層と数えるようなので、以下のように2層ニューラルネットとなる）

<br><br>
<img src = ".\img\5-34実装版のニューラルネットのイメージ.png" width="500" height="400">
<br><br>

In [1]:
#勉強のため、ファイルではなく明示的にクラスを定義する
########################各レイヤの中身を定義するところ########################
import sys, os
# 親ディレクトリのファイルをインポートするための設定
sys.path.append("C:\\Users\\satoshi\\Desktop\\DeepLearning_Study\\deep-learning-from-scratch-master")
import numpy as np
#from common.functions import *
from common.util import im2col, col2im

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size


#Affineレイヤ定義　⇒Class No:１
class Affine:
    def __init__(self, W, b):
        # 重み（W）・バイアス（b）パラメータを格納する変数
        self.W =W
        self.b = b
        
        #2次元整形した行列（テンソル対応したもの）を保存する変数と元々の行列の次元を保持する変数
        self.x = None
        self.original_x_shape = None
        
        # 重み・バイアスパラメータを偏微分した結果を格納する変数
        self.dW = None
        self.db = None

    #順伝播処理
    def forward(self, x):
        #元の次元数をバックアップ 
        self.original_x_shape = x.shape
        #テンソル対応(多次元配列の次元を一つ減らす⇒今回は意味なし？)
        x = x.reshape(x.shape[0], -1)
        #整形した行列をクラス変数として保持
        self.x = x
        #Affineレイヤの順伝播は重み付き信号の総和を伝播する
        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        ##ｐ148～P151
        #dout（前のレイヤからの逆伝播値）とwを転置したものの内積をとる=ｘ：入力の逆伝播値
        dx = np.dot(dout, self.W.T)
        #xを転置したものと、dout（前のレイヤからの逆伝播値）の内積をとる=重みの逆伝播値
        self.dW = np.dot(self.x.T, dout)
        #以前よくわからなかったところ。相変わらずよくわからん。bの逆伝播値：前レイヤを合計して行減らす
        #print("バイアス逆伝播前：",dout)
        self.db = np.sum(dout, axis=0)
        #print("バイアス逆伝播値：",self.db)
        # 入力データの形状に戻す（テンソル対応）
        dx = dx.reshape(*self.original_x_shape)  
        return dx


#Reluレイヤ定義　⇒Class No:2
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        #与えられた値が0より大きければそのまま伝播（Relu）
        self.mask = (x <= 0) #0以下のTrue/False判定
        out = x.copy() #元データリターンする変数にコピー
        out[self.mask] = 0 #Trueの箇所（0以下の箇所）を0に置き換え

        return out

    def backward(self, dout):
        #逆伝播された値が0より大きければそのまま逆伝播（Relu）
        dout[self.mask] = 0 #伝播された来た値のTrueの箇所（0以下の箇所）を0に置き換え
        dx = dout

        return dx

#Softmax&Loss（交差エントロピー誤差）レイヤ定義　⇒Class No:3
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        #教師データのベクトルの要素数を取得
        batch_size = self.t.shape[0]
        # 教師データがone-hot表現か判定（教師データ要素数=softmaxの出力数)P92
        if self.t.size == self.y.size: 
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx
########################各レイヤの中身を定義終わり########################

In [6]:
########################定義した各レイヤを使ってニューラルネットを定義########################
import sys, os
# 親ディレクトリのファイルをインポートするための設定
sys.path.append("C:\\Users\\satoshi\\Desktop\\DeepLearning_Study\\deep-learning-from-scratch-master")
import numpy as np
#理解するためにlayersファイル（各種レイヤのクラス定義）は上で定義する
#from common.layers import *　
from common.gradient import numerical_gradient #数値微分の関数を読込
from collections import OrderedDict


class TwoLayerNet:
    #インスタンス作成時に実行
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化（pramsというクラス変数を持つ）
        self.params = {}
        #第１層の重みの初期値として、784×50行列を生成（値は標準正規分布からのランダム数×学習率：0.01）
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) 
        #第１層のバイアスの初期値として、50行のベクトルを生成（値は0）
        self.params['b1'] = np.zeros(hidden_size)
        #第2層の重みの初期値として、50×10行列を生成（値は標準正規分布からのランダム数×学習率：0.01）
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        #第2層のバイアスの初期値として、10行のベクトルを生成（値は0）
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成（layersというクラス変数を持つ）
        self.layers = OrderedDict() #名前付きでデータを保存できるコレクションを定義
        #Affineクラスのインスタンスを生成　⇒Class No:１
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) 
        #Reluクラスのインスタンスを生成　⇒Class No:2
        self.layers['Relu1'] = Relu() 
        #Affineクラスのインスタンスを生成　⇒Class No:１
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2']) 
        #SoftmaxWithLossクラスのインスタンスを生成　⇒Class No:3
        self.lastLayer = SoftmaxWithLoss()
     
    #predictメソッドを定義
    def predict(self, x):
        #レイヤのコレクション分ループ
        for layer in self.layers.values():
            #各レイヤのfoward（順伝播）処理を実行
            x = layer.forward(x)
        
        return x
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
########################ニューラルネットの定義終わり########################

### 5.7.4　誤差逆伝播法を使った学習

In [7]:
########################定義したニューラルネットを使って学習を行う########################
import sys, os
# 親ディレクトリのファイルをインポートするための設定
sys.path.append("C:\\Users\\satoshi\\Desktop\\DeepLearning_Study\\deep-learning-from-scratch-master")
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

np.set_printoptions(precision=3) #有効桁３桁で丸める

# データの読み込み(normalize:入力値を0.0～1.0に正規化するか、one_hot_label：正解不正解ラベルを0・1で表現するか)
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

#x_trainの画像を確認するコード　⇒　処理が中断するのでコメントアウト。下で画像を読込
#(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True , normalize=False)
#img = x_train[0]
#print(img.shape) 
#img = img.reshape(28,28)
#print(img.shape)
#img = Image.fromarray(np.uint8(img))
#img.show()

■x_train[0]の画像を確認
<br><br>
<img src = ".\img\x_train[0].png" width="200" height="100">
<br><br>
■x_train[0]のデータを確認

In [9]:
print("=====x_train[0]=====")
print("x_train[0]の大きさ確認：",x_train[0].shape) #x_train[0]の行列の大きさを確認　⇒　1行784列
print(x_train[0]) #どんな形式で格納されているか確認 normalize=trueしているので0～1のハズ
print("=====t_train[0]=====")
print(t_train[0]) #正解ラベルが本当に５か確認
print("=====x_train=====")
print(x_train.shape) #最後にx_trainにこんなデータがどのくらいあるのかチェック⇒60,000行

=====x_train[0]=====
x_train[0]の大きさ確認： (784,)
[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0. 

In [10]:
#自作した2層レイヤのクラスをインスタンス化
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

<br><br>
<img src = ".\img\5-33実装版のニューラルネットのイメージ.png" width="500" height="400">
<br><br>

In [11]:
iters_num = 10000 #イテレーション回数　（≒epoch数）
train_size = x_train.shape[0] # トレーニングデータのデータ数（行数）
batch_size = 100 # ミニバッチのサイズ
learning_rate = 0.1 #学習率（1回の学習でどれだけパラメータを更新するか）の初期設定

train_loss_list = [] #誤差の結果を格納する箱を用意
train_acc_list = [] #トレーニングデータのaccuracyを格納する箱を用意
test_acc_list = [] #テストデータのaccuracyを格納する箱を用意

#１エポック指定：60000÷100=600　これは１epoch分（学習データを一回総なめしたタイミング）
iter_per_epoch = max(train_size / batch_size, 1)
print(iter_per_epoch)

600.0


In [12]:
#トレーニングデータ(6万件)からランダムにミニバッチ分（100件）データを取得する
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
print("ミニバッチサイズ：",batch_mask.shape)
print("ミニバッチとして取得した添え字：",batch_mask)
print(x_batch[0])
print(t_batch[0])

ミニバッチサイズ： (100,)
ミニバッチとして取得した添え字： [46094 20150  1067 39921 37927 14266 50336 18299 13596 54912 44368   656
 44041 34848 45436 13064  6924 39363 16211 58606 18796 12496 21325 38239
 31486 29165 15913 52465 47456 36249  1767 24142 42633 22087 26247   667
 11188   367 53958 37477  4626  4104 19874 11586 39809 32553 55033 21894
 59593 48251 12840   206 35632 48563 21517 15941 33283 58012 35473 28745
 52420  4506  6151 50569 37703 44979  9085  4175 18724  5583 41945 36465
 21404   855 49927 25742 38958  2243 17615 47904 42877   785  2562 14861
 10662 35221  6300 27982 37691 31480 46856 19519  9369 20182  2217 48104
 57545 22192 44610 16632]
[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.   

In [13]:
#イテレーション回数分ループ（epoch換算では、10000÷600=16～17epoch）
for i in range(iters_num):
    #トレーニングデータ(6万件)からランダムにミニバッチ分（100件）データを取得する
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #勾配を計算（TwoLayerNetクラスのgradientメソッド呼び出し）
    grad = network.gradient(x_batch, t_batch)
    
    #重みとバイアスを更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    #交差エントロピー誤差を計算（TwoLayerNetクラスのlossメソッド呼び出し）
    loss = network.loss(x_batch, t_batch)
    #誤差を格納する箱に結果を入れる
    train_loss_list.append(loss)
    
    #ループ回数が600の倍数の時（１epoch終了毎）、途中経過を表示する
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(i, train_acc, test_acc)


0 0.128383333333 0.136
600 0.904283333333 0.9065
1200 0.926333333333 0.93
1800 0.93815 0.9384
2400 0.942533333333 0.9408
3000 0.950716666667 0.9463
3600 0.956916666667 0.9531
4200 0.961416666667 0.9569
4800 0.964016666667 0.9573
5400 0.966783333333 0.9596
6000 0.97 0.9635
6600 0.97215 0.9648
7200 0.973116666667 0.9666
7800 0.97465 0.9658
8400 0.976266666667 0.9678
9000 0.978266666667 0.9695
9600 0.977966666667 0.9691


### 5.7.3　誤差逆伝播法の勾配確認

In [13]:
# coding: utf-8
import sys, os
# 親ディレクトリのファイルをインポートするための設定
sys.path.append("C:\\Users\\satoshi\\Desktop\\DeepLearning_Study\\deep-learning-from-scratch-master")
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:3.08622340559e-13
b1:1.36124966551e-12
W2:1.11549492218e-12
b2:1.19237948681e-10


## 5.8　まとめ
第5章では、視覚的に計算の過程を表す計算グラフという方法を学んだ。この計算グラフを用いて誤差逆伝播法を学び、また、ニューラルネットワークで行う処理をレイヤという単位で実装した（Affineレイヤ、Softmaxレイヤなど）。各レイヤはforwardとbackwardというメソッドが実装されており、データを順伝播、逆伝播することで重みパラメータの勾配を効率的に求めることができる。
<br>このレイヤを自由に組み合わせることで、自分の好きなネットワークを<b>簡単に？</b>作ることができる。